# Pair Automation using ```pipeline2```

## Imports and plot settings

Run this once

In [ ]:
import os
import json
import numpy as np
from matplotlib import pyplot as plt
import h5py as h5
import specpy as sp

import pipeline2 as p2

# plotting params
%matplotlib inline
# use this to change size of plots
plt.rcParams['figure.figsize'] = [6, 6]

## Code definition for a single acquisition

Run this once

In [ ]:
def run_pipeline(save_path, im, hours_to_image, ov_json, det_jsons, start_coords, sigma,
                 thresholds, ov_moves, ov_fovs=None, ov_psz=None, ov_fovs_first=None, 
                 det_fovs=None, det_pszs=None, ov_json_imspector=None, det_jsons_imspector=None,
                 onecolor=False, skip_details=False, n_reps_detail=1, sample_name=None,
                 ov_mode=None, det_modes=None):
      
    # make output dir if it does not exist already
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # init levels and name handler
    pll = p2.PipelineLevels('field', 'sted')
    name_handler = p2.DefaultNameHandler(save_path, pll)

    # abuse name_handler to get h5 filename
    # calling with empty index gives just the random hash
    h5file_name = name_handler.get_path([]).replace('.msr', '.h5')

    # open resulting h5 file in context manager
    with h5.File(h5file_name, mode='a') as h5fd:
        data_store = p2.HDF5DataStore(h5fd, pll)

        # init pipeline
        pl = (p2.AcquisitionPipeline('spot-pair-pipeline' if sample_name is None else sample_name)
                .withImspectorConnection(p2.imspector.ImspectorConnection(im).withVerbose())
                .withPipelineLevels(pll)
                .withNameHandler(name_handler)
                .withAddedStoppingCondition(p2.stoppingcriteria.TimedStoppingCriterion(hours_to_image * 60 * 60)))

        # attach h5 data store
        pl.withDataStorage(data_store)

        # overview task generator: move and hold focus
        atg_overview = (p2.taskgeneration.AcquisitionTaskGenerator(pll.field, 
                            p2.taskgeneration.DefaultLocationRemover( # 1. load overview JSON and ignore locations there
                                p2.taskgeneration.JSONFileConfigLoader(
                                    [ov_json],
                                    [ov_json_imspector] if not ov_json_imspector is None else None
                                )
                            ),
                            p2.taskgeneration.DefaultStageOffsetsSettingsGenerator( # 2. (x,y) spiral coordinates from start_pos
                                p2.taskgeneration.SpiralOffsetGenerator().withStart(start_coords).withFOV(ov_moves)
                            ),
                            p2.taskgeneration.DefaultScanModeSettingsGenerator( # 3. set overview mode
                                ['xyz' if ov_mode is None else ov_mode] , True
                            ),
                            p2.taskgeneration.DifferentFirstFOVSettingsGenerator(ov_fovs, ov_psz, ov_fovs_first), # 4. set FOV
                            p2.taskgeneration.DefaultStageOffsetsSettingsGenerator( # 5. change z for focus hold
                                p2.detection.SimpleLegacyFocusHold(
                                    p2.taskgeneration.NewestDataSelector(pl, pll.field)
                                ).withVerbose(True)
                            )
                    )
                    .withDelay(.4)) # some delay in setting parameters (e.g. for stage movement)

        # init pair detector in overview
        if not onecolor:
            detector = p2.detection.LegacySpotPairFinder(
                p2.taskgeneration.NewestDataSelector(pl, pll.field),
            sigma, thresholds).withPlotDetections(True).withVerbose()
        else:
            detector = p2.detection.SimpleSingleChannelSpotDetector(
                p2.taskgeneration.NewestDataSelector(pl, pll.field),
                sigma, thresholds[0] # use channel 0
            ).withPlotDetections(True).withVerbose()

        # detail task generation: settings from last overview + new FOV + detection
        # optionally repeat measurement to check reproducibility
        atg_detail = p2.taskgeneration.AcquisitionTaskGenerator(pll.sted,    
                                 p2.taskgeneration.DefaultLocationKeeper( # 1. take stage pos of overview
                                     p2.taskgeneration.NewestSettingsSelector(pl, pll.field)
                                 ),
                                 p2.taskgeneration.DefaultLocationRemover( # 2. load param (sets) from JSON, ignore loc
                                     p2.taskgeneration.JSONFileConfigLoader(
                                         det_jsons,
                                         det_jsons_imspector if not det_jsons_imspector is None else None,
                                         False)
                                 ),
                                 p2.taskgeneration.DefaultScanModeSettingsGenerator( # 3. set to desired modes (xy, xyz, ...)
                                     det_modes if not det_modes is None else ['xyz'] * len(det_jsons),
                                     False),
                                 p2.taskgeneration.DefaultFOVSettingsGenerator(det_fovs, det_pszs), # 4. set FOVs
                                 p2.taskgeneration.DefaultScanOffsetsSettingsGenerator(
                                     p2.detection.SimpleLocationRepeater( # 5. locations from detector (optionally repeated) 
                                         detector, n=n_reps_detail
                                     )
                                 ))
        atg_detail.withDelay(.2)

        # overview callback: re-add overview
        pl.withCallbackAtLevel(atg_overview, pll.field)
        # overview callback 2: detect & do detail
        pl.withCallbackAtLevel(atg_detail, pll.field)
        
        
        # HACK: optionally clear all pending sted images after first one has been acquired
        def clear_details(pl):
            dets = [(prio, idx, val) for (prio, idx, val) in pl.queue.queue if prio == pll.sted]
            for (prio, idx, val) in dets:
                pl.queue.queue.remove((prio, idx, val))
        # only do this if user wants to
        if skip_details:
            pl.withCallbackAtLevel(clear_details, pll.sted)
        
        
        # call overview atg once to add first measurement to pipeline
        atg_overview(pl)

        # GO
        pl.run()


## 1) initialize acquisition list and connect to Imspector

Run this to initialize the acquisition queue and connect to imspector
Re-run to clear queued acquisitions

In [ ]:
# init acq. list
acquisitions = []

# imspector object
im = sp.Imspector()

## 2) Enqueue an acquisition

Set the parameters for a new acquisition and add it to the queue.
** Acquisition will be run at the current stage coordinates**

In [ ]:
params = {}

### CHANGE ME!
params['save_path'] = 'C:/Users//RESOLFT/Desktop/AUTOMATION/59_KB_20181106/A195_K562-wt_Alive3B-594_Alive3D-594_SiR-DNA/K562_wt/raw/'


##### (OPTIONALLY) CHANGE US!

# how long to image
params['hours_to_image'] = 12

### PARAMETER FILES
# paths of the parameters files
# we can use multiple for the SETD measurement (e.g. to do both a 2d and 3d STED acq.)
params['ov_json'] = 'C:/Users/RESOLFT/Desktop/config_json/20180906_2ch_overview_standard.json'
params['det_jsons'] = [
    'C:/Users/RESOLFT/Desktop/config_json/20181108_SiR_detail_full2d_forSiR-coimaging_K562s.json',
    'C:/Users/RESOLFT/Desktop/config_json/20181108_1ch_detail_full2d_forSiR-coimaging.json'
]

### IMSPECTOR SETTINGS, optional
# paths to imspector setting files, set to None if you do not want to change settings (e.g. SLM parameters)
params['ov_json_imspector'] = None
params['det_jsons_imspector'] = None
# NB: needs to be of the same format as ov_json and det_jsons
# uncomment lines below for example (same settings for all)
#params['ov_json_imspector'] = 'C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180821_imspector_settings_default.json'
#params['det_jsons_imspector'] = [
#    'C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180821_imspector_settings_default.json',
#    'C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180821_imspector_settings_default2.json'
#]


### DETECTOR SETTINGS
# spot pair detection parameters (~expected size, thresholds for each channel)
params['sigma'] = 2
params['thresholds'] = [1.2, 1.0]
# whether to detect only in first channel (and not look for pairs)
params['onecolor'] = True

### FOV and pixel size of overviews
# NB: we can set a bigger z stack for first stack
# NB: all sizes are in meters!

# This is optional now
params['ov_fovs'] = [[5e-05, 5e-05, 0.5e-5]]
params['ov_psz'] = [[1.5e-7, 1.5e-7, 2.5e-7]]
params['ov_fovs_first'] = [[5e-05, 5e-05, 0.5e-5]]
params['ov_moves'] = [6e-5, 6e-5] # how much to move in overview spiral
# (NB: we make it larger than FOV to avoid small overlaps)
#params['det_fovs'] = [[3e-06, 3e-06, 1.5e-6]] # STED FOV
#params['det_pszs'] = [[4.5e-8, 4.5e-8, 6e-8]] # STED Pixelsize


### Debug, QC options
params['skip_details'] = True # set to True to only image 1 detail image per field
params['n_reps_detail'] = 1 # how often to repeat each detail image


### SCAN MODES (Optional)
# which scan mode (xy, xyz, ...) to use for overviews and details
# This may be None, in which case we simply use mode set in file
params['ov_mode'] = None
# e.g.
#params['ov_mode'] = 'xyz'
params['det_modes'] = ['xy', 'xyz']
# e.g.
#params['det_modes'] = ['xyz', 'xy'] 
# NB: this needs to be alist of the same size as settings for details


### Things that are set automatically

# ensure we use slashes and not backslashes
params['save_path'] = params['save_path'].replace(os.sep, '/')
# sample name, will be set automatically
params['sample_name'] = params['save_path'].strip('/').rsplit('/')[-1]
# start at current coordinates, do not change!
params['start_coords'] = p2.imspector.get_current_stage_coords(im)

# add to queue
acquisitions.append(params)

# print the currently queued acquisitions
print("""
Queued Acquisitions:
====================
""")
for ac in acquisitions:
    print(json.dumps(ac, indent=1))

## 3) Run the acquisitions

Execute cell below to run the enqueued acquisitions

In [ ]:
# go through queued acquisitions (in reverse)
# and run them
for ac in reversed(acquisitions):
    # pass Imspector as well (we excluded it until now because otherwise, the JSON print above would not work)
    ac_t = ac
    ac_t['im'] = im
    run_pipeline(**ac)

# Reset queued acquisitions
# NB: if you cancelled a run, this might not be executed,
#     make sure to clear old acquisitions manually (step 1)
acquisitions = []

## Testing code below

Ignore unless you know what you are doing ;)

In [ ]:
p = p2.AcquisitionPipeline('1')
p.queue.put(12, 1)
p.queue.put(12, 2)
p.queue.put(14, 2)

[(q) for (q) in p.queue.queue]
[(q) for (q) in p.queue.queue]

In [ ]:
with open('C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180821_imspector_settings_default2.json', 'r') as fd:
    p = json.load(fd)
    
sp.Imspector().set_parameters("", p)

In [ ]:
def foo(a, b, **kwargs):
    print(a,b)
    bar(**kwargs)
    
def bar(c):
    print(c)
    
kwargs = {'a':1, 'b':2, 'c':3}

foo(**kwargs)

In [ ]:
import enum
class PipelineType(enum.Enum):
    TWOSTEP = 1
    THREESTEP = 2

In [ ]:
PipelineType.TWOSTEP